In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import transforms
import pandas as pd
import numpy as np
import os
import pickle
import datetime

In [2]:
NUM_EPOCH = 30
NUM_CLASSES = 2
BATCH_SIZE = 64
LEARNING_RATE = 0.001

In [4]:
data_file = 'C:/Users/Hyde/Projects/Jupyter Projects/Mid-term/sound_dataset_odd.pkl'
with open(data_file, 'rb') as f:
    dataset = pickle.load(f)

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=dataset[0],
                                           batch_size=BATCH_SIZE, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=dataset[1],
                                          batch_size=BATCH_SIZE, 
                                          shuffle=False)

xx, yy = next(iter(train_loader))
print(type(xx), type(yy))

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [3]:
class NN(nn.Module):

    def __init__(self):
        super(NN, self).__init__()
        self.L1 = nn.Linear(n_in, n_units)
        self.L2 = nn.Linear(n_units, n_units)
#         self.mp = nn.MaxPool2d(2)
        self.L3 = nn.Linear(n_units, n_out)

    def forward(self, x):
        in_size = x.size(0)
        x = x.view(in_size, -1)
        x = F.relu(self.L1(x))
#         x = F.relu(self.mp(self.conv2(x)))
        x = F.relu(self.L2(x))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.L3(x)
        return F.log_softmax(x)

In [4]:
n_in = 96*125
n_units = 100
n_out = 2

In [5]:
model = NN().float()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [6]:
model.eval()

NN(
  (L1): Linear(in_features=12000, out_features=100, bias=True)
  (L2): Linear(in_features=100, out_features=100, bias=True)
  (L3): Linear(in_features=100, out_features=2, bias=True)
)

In [9]:
# Train the model
total_step = len(train_loader)
train_loss_list = []
test_loss_list = []
test_accuracy_list = []
starttime = datetime.datetime.now()
for epoch in range(NUM_EPOCH):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
#         images = images.to(device)
#         labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        labels = labels.long()
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i) % 100 == 0:
            train_loss_list.append(loss.item())
            print ('Epoch [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, NUM_EPOCH, loss.item()))
#             model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
#             with torch.no_grad():
#                 correct = 0
#                 total = 0
#                 for images, labels in test_loader:
#                     images = images.to(device)
#                     labels = labels.to(device)
#                     outputs = model(images)
#                     _, predicted = torch.max(outputs.data, 1)
#                     total += labels.size(0)
#                     correct += (predicted == labels).sum().item()

#                 print('Test Accuracy of the model on the 250 test images: {} %'.format(100 * correct / total))
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
#         data, target = Variable(data, volatile=True), Variable(target)
#         data = data.to(device)
#         target = target.to(device)
        output = model(data).double()
        # sum up batch loss
        target = target.long()
        test_loss += F.nll_loss(output, target, size_average=False).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()

    test_loss /= len(test_loader.dataset)
    test_loss_list.append(test_loss.item())
    test_accuracy_list.append((100. * correct / len(test_loader.dataset)).item())
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
#             print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
#                    .format(epoch+1, NUM_EPOCH, i+1, total_step, loss.item()))
endtime = datetime.datetime.now()
print ((endtime - starttime).seconds)

C:\Users\Hyde\anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch [1/30], Loss: 0.6688

Test set: Average loss: 0.3203, Accuracy: 180/200 (90%)


C:\Users\Hyde\anaconda3\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Epoch [2/30], Loss: 0.2240

Test set: Average loss: 0.3440, Accuracy: 180/200 (90%)

Epoch [3/30], Loss: 0.3198

Test set: Average loss: 0.2603, Accuracy: 180/200 (90%)

Epoch [4/30], Loss: 0.2435

Test set: Average loss: 0.2120, Accuracy: 180/200 (90%)

Epoch [5/30], Loss: 0.1241

Test set: Average loss: 0.1546, Accuracy: 193/200 (96%)

Epoch [6/30], Loss: 0.1528

Test set: Average loss: 0.0899, Accuracy: 194/200 (97%)

Epoch [7/30], Loss: 0.0851

Test set: Average loss: 0.0354, Accuracy: 200/200 (100%)

Epoch [8/30], Loss: 0.0277

Test set: Average loss: 0.0143, Accuracy: 200/200 (100%)

Epoch [9/30], Loss: 0.0136

Test set: Average loss: 0.0071, Accuracy: 200/200 (100%)

Epoch [10/30], Loss: 0.0070

Test set: Average loss: 0.0053, Accuracy: 200/200 (100%)

Epoch [11/30], Loss: 0.0050

Test set: Average loss: 0.0028, Accuracy: 200/200 (100%)

Epoch [12/30], Loss: 0.0017

Test set: Average loss: 0.0020, Accuracy: 200/200 (100%)

Epoch [13/30], Loss: 0.0013

Test set: Average loss: 0.

In [41]:
from plotly import graph_objs as go
from plotly.subplots import make_subplots
from _plotly_future_ import v4_subplots

In [42]:
x_list = []
for i in range(30):
    x_list.append(i)

fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.5, 0.5],
    )

fig.add_trace(
    go.Scatter(x = x_list,
                  y = train_loss_list,
                  mode = 'lines',
                  name = 'train loss'),
    row=1, col=1,
)

fig.add_trace(
    go.Scatter(x = x_list,
                  y = test_loss_list,
                  mode = 'lines',
                  name = 'test loss'),
    row=1, col=1,
)

fig.add_trace(
    go.Scatter(x = x_list,
                  y = test_accuracy_list,
                  mode = 'lines',
                  name = 'test accuracy'),
    row=1, col=2,
)

fig.show()